In [55]:
import pandas as pd # Pandas is an open source library providing data structures and data analysis tools for Python
import re #regex
from functools import reduce 
import numpy as np
from math import isnan
import matplotlib.pyplot as plt
import urllib.request
import json
from datetime import date
from datetime import datetime

#### TODAY'S DATE

In [56]:
today = '12jan'

In [95]:
beta_file_loc =  "C:/Users/huixin/OneDrive - UC San Diego/menoplan/pilot_phase/merged_analysis/"
pilot_file_loc = "C:/Users/huixin/OneDrive - UC San Diego/menoplan/trial_phase/"
cleaned = pd.read_csv(pilot_file_loc + "cleaned_data/allData" + today  + ".csv")
screening = pd.read_csv(pilot_file_loc +"qualtrics_data/" + today+ "/"+ today+ "_p1.csv")
screening=screening.iloc[2:] #drop first two rows
screening=screening.add_suffix('_p1')
qualtrics_data=qualtrics_data.iloc[2:] #drop first two rows

In [96]:
screening=screening.rename(columns={'IPAddress_p1':'IPAddress_p2', 
                                   #'ResponseId': 'ResponseId_p2'
                                   })
allD = pd.merge(screening, cleaned, on=['IPAddress_p2'], how='inner')
allD['IPFreq']=allD['IPAddress_p2'].map(allD['IPAddress_p2'].value_counts())

In [93]:
allD.to_csv(pilot_file_loc+"cleaned_data/allData"+ today+ ".csv")

In [99]:
screening

,StartDate_p1,EndDate_p1,Status_p1,IPAddress_p2,Progress_p1,Duration (in seconds)_p1,Finished_p1,RecordedDate_p1,ResponseId_p1,RecipientLastName_p1,...,BrowserInfo_Operating System_p1,BrowserInfo_Resolution_p1,Email_p1,Email2_p1,QCity_ExportTag_p1,QState_ExportTag_p1,Referer_p1,cityName_p1,Q_URL_p1,Q_BallotBoxStuffing_p1
2,2022-01-12 13:35:45,2022-01-12 13:37:10,0,172.56.14.2,100,85,1,2022-01-12 13:37:10,R_vfzPSX8176jsW2Z,NaN,...,Android 11,412x892,Georgia.craig@snhu.edu,Georgia.craig@snhu.edu,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
3,2022-01-12 14:19:41,2022-01-12 14:21:34,0,207.62.246.188,100,113,1,2022-01-12 14:21:34,R_1LFHdaW74eHPrjI,NaN,...,Windows NT 10.0,2250x1500,RaquelPuentes69@yahoo.com,RaquelPuentes69@yahoo.com,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
4,2022-01-12 15:46:02,2022-01-12 15:47:55,0,68.38.9.234,100,112,1,2022-01-12 15:47:55,R_2E9QLXT9XXDxGGD,NaN,...,Android 11,320x694,smeans082173@gmail.com,smeans082173@gmail.com,NaN,NaN,https://lm.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
5,2022-01-12 15:47:34,2022-01-12 15:48:31,0,102.135.170.122,100,56,1,2022-01-12 15:48:31,R_AtYoxtNEgIk7YVH,NaN,...,Android 11,360x820,NaN,NaN,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
6,2022-01-12 16:30:55,2022-01-12 16:35:43,0,68.66.100.37,100,287,1,2022-01-12 16:35:43,R_4YOccMPIgDbPlIZ,NaN,...,Android 11,360x780,Micberg45@gmail.com,Micberg45@gmail.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
7,2022-01-12 18:09:44,2022-01-12 18:12:16,0,71.166.115.162,100,151,1,2022-01-12 18:12:16,R_2tbB7jwDDMVllq0,NaN,...,Windows NT 10.0,1600x900,15683tb@gmail.com,15683tb@gmail.com,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
8,2022-01-12 18:14:27,2022-01-12 18:15:32,0,73.21.245.36,100,64,1,2022-01-12 18:15:32,R_UMcgsiQM0GnFQfn,NaN,...,Android 11,412x877,jenljbear@gmail.com,jenljbear@gmail.com,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
9,2022-01-12 18:47:09,2022-01-12 18:49:06,0,206.127.123.145,100,117,1,2022-01-12 18:49:07,R_0vtPXVrlyKm1eHT,NaN,...,iPhone,414x896,Christy.leeper@outlook.com,Christy.leeper@outlook.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
10,2022-01-12 19:25:10,2022-01-12 19:36:03,0,173.237.207.65,100,652,1,2022-01-12 19:36:04,R_eEU4IHbGvi1sjyV,NaN,...,Android 10,360x820,noiqee@gmail.com,noiqee@gmail.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
11,2022-01-12 19:25:35,2022-01-12 19:55:41,0,91.219.212.86,100,1806,1,2022-01-12 19:55:41,R_6L1ERSWq6juSbsJ,NaN,...,Android 9,360x800,shopeally@gmail.com,shopeally@gmail.com,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
